In [1]:
#imports
from google.cloud import vision 
import io
import os
import pandas as pd
from IPython.display import Image

#authentication
PATH_TO_CREDENTIALS = "/Users/shaydamoezzi/Desktop/keyfile.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=PATH_TO_CREDENTIALS

In [2]:
PATH_TO_IMAGES_miniplaces = '/Users/shaydamoezzi/Dropbox (MIT)/UROP/TestImageData/' #shayda's local computer path

#PATH_TO_IMAGES_miniplaces = '/Users/shaydamoezzi/Dropbox (MIT)/car-can-explain/data/TestImageData/' #change path accordingly
folders = ['bridge', 'classroom', 'coffee_shop', 'construction_site']

def object_label_annotations(path):
    """Given a batch of images returns 
    objects detected and label annotations from 
    google vision API"""
    
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response_labels = client.label_detection(image=image)
    labels = response_labels.label_annotations
    

    
    response_object = client.object_localization(image=image)
    objects = response_object.localized_object_annotations
    
    object_scores = {'Object':[], 'Score':[]}
    for object in objects:   
       object_scores['Object'].append(object.name)
       object_scores['Score'].append(object.score)


    label_scores = {'Label': [], 'Score':[]}
    for label in labels:   
       label_scores['Label'].append(label.description)
       label_scores['Score'].append(label.score)
    


    
    
    df_labels = pd.DataFrame(data=label_scores)
    df_objects = pd.DataFrame(data=object_scores)
   
    return (df_labels, df_objects)

label_list = []
object_list = []
name_list = []

for folder in folders:
    for file in os.listdir(PATH_TO_IMAGES_miniplaces+folder+'/'):
        filename = PATH_TO_IMAGES_miniplaces+folder+'/'+file
        labels = object_label_annotations(filename)[0]
        objects = object_label_annotations(filename)[1]
        base_filename = 'Values.txt'
        name_list.append(file)
        label_list.append(labels.to_string())
        object_list.append(objects.to_string())

        

file = open('/Users/shaydamoezzi/Dropbox (MIT)/UROP/labels','w')
for i in range(len(label_list)):
    file.write(name_list[i]+ '\n'+ label_list[i] + '\n'+ object_list[i] + '\n' +'\n')
file.close()
            
